# Seq2Seq 실습 1: 영어-한국어 번역


### 1. 환경 설정 및 라이브러리 설치

* `torch`: 딥러닝 프레임워크
* `torchtext`: 텍스트 처리를 위한 라이브러리
* `spacy`: 영어 토큰화
* `konlpy`: 한국어 토큰화 (Okt 사용)

In [1]:
# Colab 셀 1: 라이브러리 설치
# torch 및 torchtext의 호환되는 버전을 명시하여 설치
# !pip uninstall -y torch torchtext torchvision torchaudio
!pip install --upgrade torch==2.3.0 torchvision==0.18.0 torchaudio==2.3.0 torchtext==0.18.0
!pip install spacy konlpy
!python -m spacy download en_core_web_sm

# konlpy 사용을 위한 자바 설치
!apt-get install openjdk-8-jdk -y

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 131.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  ca-certificates-java fonts-dejavu-core fonts-dejavu-extra java-common
  libatk-wrapper-java libatk-wrapper-java-jni libgail-common libgail18
  libgtk2.0-0 libgtk2.0-bin libgtk2.0-common libpcsclite1 librsvg2-common
  libxt-dev libxtst6 libxxf86dga1 openjdk-8-jdk-headless openjdk-8-jre
  openjdk-8-jre-headless x11-utils
Suggested packages:
  default-jre gvfs pcscd libxt-doc openjdk-8-demo openjdk-8-source visualvm
  libnss-mdns 

In [2]:
import torch
import torchtext

print(torch.__version__)
print(torchtext.__version__)

2.3.0+cu121
0.18.0+cpu


In [3]:
# 1) JPype 관련 패키지 싹 제거
!pip uninstall -y jpype JPype1 jpype1

# 2) 자바 설치 (Colab용, JDK 17)
!apt-get -y update
!apt-get -y install openjdk-17-jdk-headless

# 3) JPype1, konlpy 깔끔하게 재설치
!pip install JPype1==1.4.1
!pip install konlpy

Found existing installation: jpype1 1.6.0
Uninstalling jpype1-1.6.0:
  Successfully uninstalled jpype1-1.6.0
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://cli.github.com/packages stable InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,123 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:12 https://ppa.launchpadcon

### 2. 데이터 다운로드 및 전처리

[Tatoeba 프로젝트](http://www.manythings.org/anki/)의 소규모 영-한 병렬 코퍼스를 사용

In [4]:
# Colab 셀 2: 데이터 다운로드 및 압축 해제
!wget http://www.manythings.org/anki/kor-eng.zip
!unzip kor-eng.zip

--2025-11-10 01:27:30--  http://www.manythings.org/anki/kor-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 253046 (247K) [application/zip]
Saving to: ‘kor-eng.zip’

kor-eng.zip         100%[===================>] 247.12K  --.-KB/s    in 0.1s    

2025-11-10 01:27:30 (1.61 MB/s) - ‘kor-eng.zip’ saved [253046/253046]

Archive:  kor-eng.zip
  inflating: _about.txt              
  inflating: kor.txt                 


### 3. 기본 라이브러리 임포트 및 설정

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from torchtext.vocab import build_vocab_from_iterator

import spacy
from konlpy.tag import Okt          # 한국어 형태소 분석기(예: "사과를 먹었다">>['사과', '를', '먹었', '다']

import random
import math
import time
import io
import os

# 딥러닝이나 머신러닝에서는 모델 학습 과정에 무작위(random) 요소 많음: 가중치, 셔플, dropout ...
# 매번 같은 결과를 재현위해서 시드 고정: “같은 코드 + 같은 데이터 + 같은 시드 → 같은 결과!”

SEED = 1234
random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

/usr/local/lib/python3.12/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.12/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


Using device: cuda


### 4. 토크나이저 정의

영어는 `spaCy`, 한국어는 `Okt`를 사용해 토크나이저를 정의

In [22]:
# 토크나이저 로드 및 정의
import spacy
from konlpy.tag import Okt

spacy_en = spacy.load('en_core_web_sm')
okt = Okt()

def tokenize_en(text):
  """
  spaCy를 사용한 영어 토크나이저
  """
  return [tok.text for tok in spacy_en.tokenizer(text)]

def tokenize_ko(text):
  """
  Okt를 사용한 한국어 토크나이저
  """
  return okt.morphs(text)

# 테스트
print(tokenize_en("Hello, my name is Kim."))
print(tokenize_ko("안녕하세요, 제 이름은 김입니다."))

['Hello', ',', 'my', 'name', 'is', 'Kim', '.']
['안녕하세요', ',', '제', '이름', '은', '김', '입니다', '.']


### 5. 데이터 로드 및 어휘장(Vocabulary) 구축

다운로드한 `kor.txt` 파일(탭으로 구분)을 읽고, 영어와 한국어 어휘장을 만듦

In [23]:
import io

# 데이터 로드 및 어휘장 구축
DATA_PATH = 'kor.txt'
MIN_FREQ = 5   #단어장에 포함될 최소 빈도

# 특수 토큰 정의
UNK_IDX, PAD_IDX, SOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<sos>', '<eos>']

# 영어/한국어 토큰 제너레이터
def yield_en_tokens(data_path, tokenizer_en):
  with io.open(data_path, encoding = 'utf-8') as f:
    for line in f:
      parts = line.strip().split('\t')
      if len(parts) >= 2:
        yield tokenizer_en(parts[0])

def yield_ko_tokens(data_path, tokenizer_ko):
  with io.open(data_path, encoding = 'utf-8') as f:
    for line in f:
      parts = line.strip().split('\t')
      if len(parts) >= 2:
        yield tokenizer_ko(parts[1])

from torchtext.vocab import build_vocab_from_iterator

print("Building English vocab...")
vocab_en = build_vocab_from_iterator(
    yield_en_tokens(DATA_PATH, tokenize_en),
    min_freq=MIN_FREQ,
    specials=special_symbols,
    special_first=True
)
vocab_en.set_default_index(UNK_IDX) # <unk> 토큰의 인덱스를 기본값으로 설정

print("Building Korean vocab...")
vocab_ko = build_vocab_from_iterator(
    yield_ko_tokens(DATA_PATH, tokenize_ko),
    min_freq=MIN_FREQ,
    specials=special_symbols,
    special_first=True
)
vocab_ko.set_default_index(UNK_IDX)

print(f"English Vocab Size: {len(vocab_en)}")
print(f"Korean Vocab Size: {len(vocab_ko)}")

Building English vocab...
Building Korean vocab...
English Vocab Size: 918
Korean Vocab Size: 1014


### 6. Dataset 및 DataLoader 정의

PyTorch의 `Dataset`과 `DataLoader`를 사용하여 데이터를 배치 단위로 처리할 수 있도록 준비합니다.

In [24]:
# Dataset 정의
from torch.utils.data import Dataset, DataLoader

class EngKorDataset(Dataset):
  def __init__(self, data_path, vocab_en, vocab_ko, tokenizer_en, tokenizer_ko):
    self.vocab_en = vocab_en
    self.vocab_ko = vocab_ko
    self.tokenizer_en = tokenizer_en
    self.tokenizer_ko = tokenizer_ko
    self.data = []

    with io.open(data_path, encoding='utf-8') as f:
      for line in f:
        parts = line.strip().split('\t')
        if len(parts) >= 2:
          # [영어, 한국어] 쌍으로 저장
          self.data.append((parts[0], parts[1]))

  def __len__(self):
      return len(self.data)

  def __getitem__(self, idx):
      eng_text, kor_text = self.data[idx]
      return eng_text, kor_text

In [25]:
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader

# Define device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


# Collate 함수 및 DataLoader 생성
def collate_fn(batch):
  """
    PyTorch의 DataLoader는 여러 샘플을 묶어 **하나의 배치(batch)**로 만듭니다.
    하지만 NLP에서는 문장의 길이가 제각각이라 단순히 묶기 어렵습니다.
    collate_fn은 각 문장을 토큰화 → 인덱스 변환 → 패딩(padding) 해서 동일한 길이로 만들어주는 함수
  """
  src_batch, trg_batch = [], []
  for src_sample, trg_sample in batch:
      # 1. 토큰화
      src_tokens = [tok for tok in tokenize_en(src_sample.lower())]
      trg_tokens = [tok for tok in tokenize_ko(trg_sample)]

      # 2. <sos> 및 <eos> 추가
      src_with_eos = src_tokens + ['<eos>']
      trg_with_sos_eos = ['<sos>'] + trg_tokens + ['<eos>']

      # 3. 수치화 (Vocab 사용)
      src_indices = [vocab_en[token] for token in src_with_eos]
      trg_indices = [vocab_ko[token] for token in trg_with_sos_eos]

      src_batch.append(torch.tensor(src_indices, dtype=torch.long))
      trg_batch.append(torch.tensor(trg_indices, dtype=torch.long))

  # 4. 패딩: RNN은 가변 길이 시퀀스를 받지만, 배치는 고정 길이여야 함
  # pad_sequence는 (seq_len, batch_size) 형태의 텐서를 만듦
  src_padded = pad_sequence(src_batch, padding_value=PAD_IDX)
  trg_padded = pad_sequence(trg_batch, padding_value=PAD_IDX)

  return src_padded.to(device), trg_padded.to(device)


# 데이터셋 분리 (학습 / 검증 / 테스트)
full_dataset = EngKorDataset(DATA_PATH, vocab_en, vocab_ko, tokenize_en, tokenize_ko)
train_size = int(0.8 * len(full_dataset))
valid_size = len(full_dataset) - train_size
# Test 셋은 편의상 Valid 셋과 동일하게 사용 (원래는 분리해야 함)
train_dataset, valid_dataset = torch.utils.data.random_split(full_dataset, [train_size, valid_size])

BATCH_SIZE = 128

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)

# 데이터로더 테스트
src_batch, trg_batch = next(iter(train_loader))
print(f"Source batch shape: {src_batch.shape}") # (src_len, batch_size)
print(f"Target batch shape: {trg_batch.shape})") # (trg_len, batch_size)

Using device: cuda
Source batch shape: torch.Size([14, 128])
Target batch shape: torch.Size([17, 128]))


### 7. Seq2Seq 모델 정의

#### 7.1. Encoder (인코더)

인코더는 입력 문장(영어)을 받아 하나의 '컨텍스트 벡터'(Context Vector)로 압축

In [54]:
import torch.nn as nn

# Encoder 정의
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.hid_dim = hid_dim
        self.n_layers = n_layers

        # 1. 임베딩 층
        self.embedding = nn.Embedding(input_dim, emb_dim)

        # 2. RNN 층 (GRU 사용)
        self.rnn = nn.GRU(emb_dim, hid_dim, n_layers, dropout=dropout)

        # 3. 드롭아웃
        self.dropout = nn.Dropout(dropout)

    def forward(self, src):
        embedded = self.dropout(self.embedding(src))

        outputs, hidden = self.rnn(embedded)

        return hidden


#### 7.2. Decoder (디코더)

디코더는 인코더의 컨텍스트 벡터와 이전 타임스텝의 예측 단어를 받아 다음 단어를 예측

In [62]:
# Decoder 정의
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers

        # 1. 임베딩 층
        self.embedding = nn.Embedding(output_dim, emb_dim)

        # 2. RNN 층 (GRU 사용)
        # 인코더와 동일한 파라미터를 가져야 함
        self.rnn = nn.GRU(emb_dim, hid_dim, n_layers, dropout=dropout)

        # 3. Fully Connected (Linear) 층
        # RNN의 hidden state를 받아 어휘장 크기의 벡터로 변환
        self.fc_out = nn.Linear(hid_dim, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, input, hidden):
        # input: (batch_size) - 현재 time-step의 입력 단어 (t)
        # hidden: (n_layers, batch_size, hid_dim) - 이전 time-step의 hidden state (t-1)

        # 1. 입력 단어 임베딩
        # (batch_size) -> (1, batch_size)로 변환 (GRU는 seq_len 차원이 필요)
        input = input.unsqueeze(0)
        # input: (1, batch_size)

        embedded = self.dropout(self.embedding(input))
        # embedded: (1, batch_size, emb_dim)

        # 2. RNN 통과
        # 디코더는 매 스텝마다 컨텍스트 벡터(hidden)를 업데이트
        output, hidden = self.rnn(embedded, hidden)

        # output: (1, batch_size, hid_dim)
        # hidden: (n_layers, batch_size, hid_dim)

        # 3. 예측 (Linear 층)
        # (1, batch_size, hid_dim) -> (batch_size, output_dim)
        prediction = self.fc_out(output.squeeze(0))
        # prediction: (batch_size, output_dim) - 한국어 어휘장 크기의 로짓(logit)

        return prediction, hidden

#### 7.3. Seq2Seq (모델 결합)

인코더와 디코더를 결합하고, 'Teacher Forcing'을 구현합니다.

In [63]:
# Seq2Seq 모델 정의
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        # 인코더와 디코더의 hidden dim이 일치해야 함
        assert encoder.hid_dim == decoder.hid_dim, "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, "Encoder and decoder must have same number of layers!"

    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim


        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)

        hidden = self.encoder(src)

        input = trg[0, :]
        for t in range(1, trg_len):

            output, hidden = self.decoder(input, hidden)

            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio

            top1 = output.argmax(1)
            input = trg[t] if teacher_force else top1

        return outputs

### 8. 모델 학습

#### 8.1. 하이퍼파라미터 및 모델 초기화

In [64]:
import torch.optim as optim
import torch.nn as nn # Add this import for CrossEntropyLoss
import torch # Add this import for torch.cuda.is_available() and torch.manual_seed

# 모델 파라미터 및 초기화
INPUT_DIM = len(vocab_en)
OUTPUT_DIM = len(vocab_ko)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

# 모델 인스턴스 생성
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)
model = Seq2Seq(enc, dec, device).to(device)

# 파라미터 수 확인
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {count_parameters(model):,} trainable parameters')

# 옵티마이저 (Adam)
optimizer = optim.Adam(model.parameters())

# 손실 함수 (CrossEntropyLoss)
# <pad> 토큰(인덱스 1)은 무시하도록 ignore_index 설정
criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

The model has 6,532,086 trainable parameters


#### 8.2. 학습 및 평가 함수 정의

In [65]:
# 학습 함수
def train(model, iterator, optimizer, criterion, clip):
    model.train() # 학습 모드
    epoch_loss = 0

    for i, (src, trg) in enumerate(iterator):
        optimizer.zero_grad()

        # 1. 모델 순전파
        # teacher_forcing_ratio = 0.5 (50% 확률로 Teacher Forcing 사용)
        output = model(src, trg, 0.5)

        # output: (trg_len, batch_size, output_dim)
        # trg: (trg_len, batch_size)

        output_dim = output.shape[-1]

        # 2. 손실 계산
        # CrossEntropyLoss는 (N, C) 형태의 2D 입력과 (N) 형태의 1D 타겟을 기대함
        # <sos> 토큰(인덱스 0)은 예측 대상이 아니므로 [1:] 부터 사용
        output = output[1:].view(-1, output_dim) # ( (trg_len-1) * batch_size, output_dim )
        trg = trg[1:].view(-1)                   # ( (trg_len-1) * batch_size )

        loss = criterion(output, trg)

        # 3. 역전파 및 가중치 업데이트
        loss.backward()

        # 4. Gradient Clipping (폭주 방지)
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()
        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [66]:
# 평가 함수
def evaluate(model, iterator, criterion):
    model.eval() # 평가 모드
    epoch_loss = 0

    with torch.no_grad(): # 그래디언트 계산 비활성화
        for i, (src, trg) in enumerate(iterator):

            # 1. 모델 순전파 (평가 시에는 Teacher Forcing 사용 안 함)
            output = model(src, trg, 0) # teacher_forcing_ratio = 0

            # 2. 손실 계산 (마찬가지로 <sos> 토큰 제외)
            output_dim = output.shape[-1]
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

#### 8.3. 실제 학습 실행 : 약 2분 시간 걸림


In [67]:
import time
import math
import torch
import random

# 학습 실행
N_EPOCHS = 50
CLIP = 1
best_valid_loss = float('inf')

# 시간 측정용
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

for epoch in range(N_EPOCHS):
    start_time = time.time()

    train_loss = train(model, train_loader, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_loader, criterion)

    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    # Validation loss가 가장 낮은 모델을 저장
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'seq2seq-model.pt')

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 10s
	Train Loss: 4.531 | Train PPL:  92.877
	 Val. Loss: 4.346 |  Val. PPL:  77.183
Epoch: 02 | Time: 0m 9s
	Train Loss: 4.075 | Train PPL:  58.855
	 Val. Loss: 4.208 |  Val. PPL:  67.240
Epoch: 03 | Time: 0m 8s
	Train Loss: 3.799 | Train PPL:  44.643
	 Val. Loss: 4.005 |  Val. PPL:  54.876
Epoch: 04 | Time: 0m 10s
	Train Loss: 3.622 | Train PPL:  37.407
	 Val. Loss: 3.983 |  Val. PPL:  53.700
Epoch: 05 | Time: 0m 9s
	Train Loss: 3.484 | Train PPL:  32.593
	 Val. Loss: 3.838 |  Val. PPL:  46.420
Epoch: 06 | Time: 0m 11s
	Train Loss: 3.340 | Train PPL:  28.232
	 Val. Loss: 3.843 |  Val. PPL:  46.683
Epoch: 07 | Time: 0m 10s
	Train Loss: 3.226 | Train PPL:  25.167
	 Val. Loss: 3.764 |  Val. PPL:  43.100
Epoch: 08 | Time: 0m 11s
	Train Loss: 3.077 | Train PPL:  21.694
	 Val. Loss: 3.760 |  Val. PPL:  42.936
Epoch: 09 | Time: 0m 11s
	Train Loss: 2.984 | Train PPL:  19.770
	 Val. Loss: 3.791 |  Val. PPL:  44.300
Epoch: 10 | Time: 0m 11s
	Train Loss: 2.886 | Train PPL:  

### 9. 결과: 모델 추론 (번역)

학습된 모델을 사용하여 새로운 영어 문장을 한국어로 번역하는 함수를 만듭니다.

In [67]:
# 번역 추론 함수
def translate_sentance(sentence, model, vocab_en, vocab_ko, tokenizer_en, device, max_len=50):
    model.eval()

    # 입력문장 토큰화 및 전처리
    tokens = [token.lower() for token in tokenizer_en(sentence)]
    tokens = tokens + ['<eos>']
    src_indexes = [vocab_en[token] for token in tokens]

    # (src_len) -> (src_len, 1) : 배치 크기 1
    src_tensor = torch.LongTensor(src_indexes).unsqueeze(1).to(device)

    # 인코더
    with torch.no_grad():
        hidden = model.encoder(src_tensor)

    # 디코더
    trg_indexes = [SOS_IDX]

    for i in range(max_len):
        # (1) -> (1, 1) : 배치 크기 1
        trg_tensor = torch.LongTensor([trg_indexes[-1]]).to(device)

